In [1]:
import dash
import dash as dcc
import dash as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import dash_bootstrap_components as dbc
import dash_mantine_components as dmc
from datetime import date, datetime
from datetime import datetime as dt

In [2]:
concated2020 = pd.read_csv("concated_mainyear_2020.csv").reset_index(drop=True)
concated2020 = concated2020.iloc[:, 1:] 
concated2021 = pd.read_csv("concated_mainyear_2021.csv").reset_index(drop=True)
concated2021 = concated2021.iloc[:, 1:] 
concated2022 = pd.read_csv("concated_mainyear_2022.csv").reset_index(drop=True)
concated2022 = concated2022.iloc[:, 1:] 
concated2023 = pd.read_csv("concated_mainyear_2023.csv").reset_index(drop=True)
concated2023 = concated2023.iloc[:, 1:] 
df = pd.concat([concated2020,
               concated2021,
               concated2022,
               concated2023],
              ignore_index=True)

In [3]:
newdf = df.copy()
newdf['Tarih'] = pd.to_datetime(df['Tarih'], format="%d.%m.%Y")

newdf['Fon Kodu'] = df['Fon Kodu'].astype(str)
newdf['Fon Adı'] = df['Fon Adı'].astype(str)

newdf['Fiyat'] = pd.to_numeric(df['Fiyat'].astype(str).str.replace(",", ""))
#newdf['Fiyat'] = newdf.apply(lambda x: "{:,}".format(x['Fiyat']), axis=1)

newdf['Tedavüldeki Pay Sayısı'] = pd.to_numeric(df['Tedavüldeki Pay Sayısı'].astype(str).str.replace(",", ""))
#newdf['Tedavüldeki Pay Sayısı'] = newdf.apply(lambda x: "{:,}".format(x['Tedavüldeki Pay Sayısı']), axis=1)

newdf['Fon Toplam Değer'] = pd.to_numeric(df['Fon Toplam Değer'].astype(str).str.replace(",", ""))
#newdf['Fon Toplam Değer'] = newdf.apply(lambda x: "{:,}".format(x['Fon Toplam Değer']), axis=1)

newdf['Kişi Sayısı'] = pd.to_numeric(df['Kişi Sayısı'].astype(str).str.replace(",", ""))
#newdf['Kişi Sayısı'] = newdf.apply(lambda x: "{:,}".format(x['Kişi Sayısı']), axis=1)

newdf = newdf.sort_values(by=['Fon Kodu', 'Tarih'])
newdf['Günlük Getiri'] = newdf.groupby('Fon Kodu')['Fiyat'].pct_change()
# newdf['Günlük Getiri2'] = (newdf['Fiyat'] / newdf.groupby('Fon Kodu')['Fiyat'].shift(1)) - 1 
## pct change komutu aynı işlevi yaptığı için iptal ettim.
newdf['Günlük Para Hareketi'] = newdf['Fon Toplam Değer'] - ((1 + newdf['Günlük Getiri']) * newdf.groupby('Fon Kodu')['Fon Toplam Değer'].shift(1))
newdf = newdf.sort_values(by=["Tarih","Fon Kodu"]).reset_index(drop=True)

kurlar = pd.read_csv("kurlar.csv")
kurlar = kurlar.iloc[:, 1:] 
kurlar["Tarih"] = pd.to_datetime(kurlar["Tarih"])
kurlar = kurlar.loc[kurlar["Kod"]=="USD"]

newdf = pd.merge(newdf, kurlar[["Tarih","ForexBuying"]], on='Tarih', how='left').rename(columns={"ForexBuying":"ForexBuyingUSD"})
newdf['FiyatUSD'] = newdf['Fiyat'] / newdf['ForexBuyingUSD']

# https://tr.investing.com/currencies/gau-try-historical-data
altin = pd.read_csv("altin.csv")
altin["Tarih"] = pd.to_datetime(altin["Tarih"])
newdf = pd.merge(newdf, altin[["Tarih","Gram Altın Alış Fiyatı"]], on='Tarih', how='left').rename(columns={"Gram Altın Alış Fiyatı":"Gram Altın Fiyatı TL"})

# https://tr.investing.com/currencies/xagg-try-historical-data
gumus = pd.read_csv("gumus.csv")
gumus["Tarih"] = pd.to_datetime(gumus["Tarih"],dayfirst=True)
gumus["Düşük"] = pd.to_numeric(gumus["Düşük"].str.replace(",","."))
gumus = gumus[["Tarih","Düşük"]].rename(columns={"Düşük":"Gram Gümüş Alış Fiyatı"})
newdf = pd.merge(newdf, gumus[["Tarih","Gram Gümüş Alış Fiyatı"]], on='Tarih', how='left').rename(columns={"Gram Gümüş Alış Fiyatı":"Gram Gümüş Fiyatı TL"})

# https://tr.investing.com/indices/ise-100-historical-data
bist100 = pd.read_csv("bist100.csv")
bist100["Tarih"] = pd.to_datetime(bist100["Tarih"],dayfirst=True)
bist100["Düşük"] = pd.to_numeric(bist100["Düşük"].str.replace(".","").str.replace(",","."))
bist100 = bist100[["Tarih","Düşük"]].rename(columns={"Düşük":"Bist 100 Fiyatı"})
newdf = pd.merge(newdf, bist100[["Tarih","Bist 100 Fiyatı"]], on='Tarih', how='left').rename(columns={"Bist 100 Fiyatı":"Bist 100 Fiyatı TL"})

# https://tr.investing.com/indices/ise-30-historical-data
bist30 = pd.read_csv("bist30.csv")
bist30["Tarih"] = pd.to_datetime(bist30["Tarih"],dayfirst=True)
bist30["Düşük"] = pd.to_numeric(bist30["Düşük"].str.replace(".","").str.replace(",","."))
bist30 = bist30[["Tarih","Düşük"]].rename(columns={"Düşük":"Bist 30 Fiyatı"})
newdf = pd.merge(newdf, bist30[["Tarih","Bist 30 Fiyatı"]], on='Tarih', how='left').rename(columns={"Bist 30 Fiyatı":"Bist 30 Fiyatı TL"})

# mevduat
mevduat = pd.read_excel("mevduat.xlsx")
mevduat = mevduat.loc[mevduat["Tarih"] >= "2020-01-01"]
mevduat["Tarih"] = pd.to_datetime(mevduat["Tarih"],dayfirst=True)
mevduat["Mevduat"] = pd.to_numeric(mevduat["Mevduat"])
newdf = pd.merge(newdf, mevduat[["Tarih","Mevduat"]], on='Tarih', how='left')


c:\users\emrecan.ulu.desktop-4t87dph\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:49: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
c:\users\emrecan.ulu.desktop-4t87dph\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:56: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [4]:
kurlar = kurlar.rename(columns={"ForexBuying": "Fiyat", "Kod": "Fon Kodu", "Isim": "Fon Adı"})
kurlar["Fon Kodu"] = kurlar["Fon Kodu"].str.replace("USD", "USDX")

altin = altin.rename(columns={"Gram Altın Alış Fiyatı": "Fiyat"})
altin["Fon Kodu"] = "ALTINX"
altin["Fon Adı"] = "Gram Altın Fiyatı TL"

gumus = gumus.rename(columns={"Gram Gümüş Alış Fiyatı": "Fiyat"})
gumus["Fon Kodu"] = "GUMUSX"
gumus["Fon Adı"] = "Gram Gümüş Fiyatı TL"

bist100 = bist100.rename(columns={"Bist 100 Fiyatı": "Fiyat"})
bist100["Fon Kodu"] = "BIST100X"
bist100["Fon Adı"] = "BIST100 Fiyatı TL"

bist30 = bist30.rename(columns={"Bist 30 Fiyatı": "Fiyat"})
bist30["Fon Kodu"] = "BIST30X"
bist30["Fon Adı"] = "BIST30 Fiyatı TL"

mevduat = mevduat.rename(columns={"Mevduat": "Fiyat"})
mevduat["Fon Kodu"] = "MEVDUATX"
mevduat["Fon Adı"] = "Mevduat Oranı TL"

In [ ]:
newdf = pd.concat([newdf,kurlar,altin,gumus,bist100,bist30,mevduat],axis=0)
newdf.to_excel("newdf.xlsx")
newdf

In [4]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Dropdown for selecting "Fon"
dropdown = dcc.Dropdown(
    id='fon-dropdown',
    options=[
        {'label': f'{fon_kodu} - {fon_adi}', 'value': fon_kodu} for fon_kodu, fon_adi in zip(newdf['Fon Kodu'].unique(), newdf['Fon Adı'].unique())
    ],
    value=newdf['Fon Kodu'].unique()[0]  # Set default value
)

# Assuming 'newdf' is your DataFrame
start_date_default = datetime(2023, 1, 1)

date_picker_range = dcc.DatePickerRange(
    id='date-picker-range',
    start_date=start_date_default,
    end_date=newdf['Tarih'].max(),
    display_format='YYYY-MM-DD'
)

fund_price_plot = dcc.Graph(figure={}, id='fund_price_plot')
kazanc_plot = dcc.Graph(figure={}, id='kazanc_plot')


# Assuming 'giris_miktari' is defined somewhere in your code
giris_miktari = 0  # Replace 0 with the initial value you want

app.layout = dbc.Container(
    fluid=True,
    children=[
        dbc.Row([
            dbc.Col(dropdown, width=8),
            dbc.Col(date_picker_range, width=2),
            dbc.Col(html.Div([
                dcc.Input(id='giris-input', type='number', placeholder='Enter amount')
            ]), width=2)
        ]),
        dbc.Row([
            dbc.Col(fund_price_plot, width=12),
        ]),
        dbc.Row([
            dbc.Col(kazanc_plot, width=12),
        ])
    ]
)

# Callback to update the graph based on the selected "Fon" and date range
@app.callback(
    Output('fund_price_plot', 'figure'),
    [Input('fon-dropdown', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date')]  # Added input for 'giris-input'
)
def update_graph(selected_fon, start_date, end_date):
    # Convert giris_miktari to float
    # Filter data based on the selected "Fon" and date range
    filtered_data = newdf.loc[(newdf['Fon Kodu'] == selected_fon) & 
                          (newdf['Tarih'] >= start_date) & 
                          (newdf['Tarih'] <= end_date)]
        
    # Create an empty figure
    fig = go.Figure()

    # Add trace for 'Fiyat'
    fig.add_trace(go.Scatter(x=filtered_data['Tarih'], y=filtered_data['Fiyat'], mode='lines', name='Fund Price in TL', line=dict(color='blue')))
    
    # Update layout to include y2 axis
    fig.update_layout(
        xaxis=dict(title='Tarih'),
        yaxis=dict(title='Fund Price in TL'),
        legend=dict(x=0, y=1)
    )

    print(giris_miktari)

    return fig

@app.callback(
    Output('kazanc_plot', 'figure'),
    [Input('fon-dropdown', 'value'),
     Input('date-picker-range', 'start_date'),
     Input('date-picker-range', 'end_date'),
     Input('giris-input', 'value')]  # Added input for 'giris-input'
)
def update_graph(selected_fon, start_date, end_date, giris_miktari):
    # Convert giris_miktari to float
    giris_miktari = float(giris_miktari) if giris_miktari else 0.0

    # Filter data based on the selected "Fon" and date range
    filtered_data = newdf.loc[(newdf['Fon Kodu'] == selected_fon) & 
                          (newdf['Tarih'] >= start_date) & 
                          (newdf['Tarih'] <= end_date)]
    fon_fiyati = filtered_data["Fiyat"].values[0]
    dolar_fiyati = filtered_data['ForexBuyingUSD'].values[0]
        
    fon_miktari = round(giris_miktari / fon_fiyati)
    dolar_miktari = giris_miktari / dolar_fiyati
    filtered_data.loc[:,"FiyatXFonMiktari"] = filtered_data.loc[:,"Fiyat"] * fon_miktari
    filtered_data.loc[:,"FiyatXDolarMiktari"] = filtered_data.loc[:,"ForexBuyingUSD"] * dolar_miktari

    # Create an empty figure
    fig = go.Figure()

    # Add trace for 'Fiyat'
    fig.add_trace(go.Scatter(x=filtered_data['Tarih'], y=filtered_data['FiyatXFonMiktari'], mode='lines', name=f'{selected_fon}', line=dict(color='red')))
    fig.add_trace(go.Scatter(x=filtered_data['Tarih'], y=filtered_data['FiyatXDolarMiktari'], mode='lines', name='USD', line=dict(color='green')))
    
    # Update layout to include y2 axis
    fig.update_layout(
        xaxis=dict(title='Tarih'),
        yaxis=dict(title=f'{selected_fon}'),
        yaxis2=dict(title='USD', overlaying='y', side='right'),
        legend=dict(x=0, y=1)
    )

    print(giris_miktari)

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)
    print(f'Dash app is running at http://127.0.0.1:8051/')


Dash app is running at http://127.0.0.1:8051/


In [ ]:
import pandas as pd

def get_fund_price(df, tarih, fon_kodu):
    try:
        return df[(df['Tarih'] == tarih) & (df['Fon Kodu'] == fon_kodu)]['Fiyat'].values[0]
    except IndexError:
        print("Hata: Belirtilen tarih ve fon kodu bulunamadı.")
        return None

def ne_kadar_kazanirdim(newdf):
    try:
        giris_miktari = float(input("Gireceğiniz para miktarını girin: "))
        giris_tarihi = input("Giriş tarihini girin (y-m-d formatında): ")
        cikis_tarihi = input("Çıkış tarihini girin (y-m-d formatında): ")
        fon_kodu = input("Almak istediğiniz fonun kodunu girin: ")

        fon_fiyati = get_fund_price(newdf, giris_tarihi, fon_kodu)
        if fon_fiyati is None:
            return
        dolar_fiyati = newdf[newdf['Tarih'] == giris_tarihi]['ForexBuyingUSD'].values[0]
        
        fon_miktari = round(giris_miktari / fon_fiyati)
        dolar_miktari = giris_miktari / dolar_fiyati

        cikis_fon_fiyati = get_fund_price(newdf, cikis_tarihi, fon_kodu)
        if cikis_fon_fiyati is None:
            return
        cikis_dolar_fiyati = newdf[newdf['Tarih'] == cikis_tarihi]['ForexBuyingUSD'].values[0]

        # Kazancı hesapla
        kazanc_fon = round(fon_miktari * cikis_fon_fiyati, 2)
        kazanc_dolar = round(dolar_miktari * cikis_dolar_fiyati, 2)

        # Sonuçları yazdır
        print("\nAldığın Fon Miktarı:", fon_miktari)
        print("Aldığın Dolar Miktarı:", round(dolar_miktari, 2))
        print(f"Paran {fon_kodu} Fonunda {kazanc_fon} TL Olmuştu.")
        print(f"Paran Dolar Cinsinde {kazanc_dolar} TL Olmuştu.")

    except ValueError:
        print("Hata: Geçersiz giriş. Lütfen doğru formatlarda veri girin.")

ne_kadar_kazanirdim(newdf)
